In [1]:
import sys, datetime
sys.path.append("../../scripts/")
from s3_support import *

import pandas as pd
import numpy as np

In [2]:
org_year = pd.read_csv("org_year.csv")

In [3]:
org_year.head()

,org,year,donors_unique,donors_retained,retention,rank
0,390147,2016,233,31,0.133047,50k
1,390147,2017,319,58,0.181818,50k
2,390147,2018,182,41,0.225275,50k
3,390147,2019,76,12,0.157895,50k
4,390147,2020,166,4,0.024096,50k


# emails

In [4]:
q = "select id, org, created, message from logs where message like '%email%'"
df_logs = redshift_query_read(q, schema='production')

q = "select id, org, created, message from syslog_logs where message like '%email%'"
df_syslogs = redshift_query_read(q, schema='production')

df_logs = df_logs.append(df_syslogs)

In [7]:
df_logs['year'] = pd.to_datetime(df_logs['created']).dt.year
logs_agg = df_logs.groupby(['org', 'year'])['id'].count().reset_index()
logs_agg.columns = ['org', 'year', 'message_logs']

In [9]:
org_year = org_year.merge(logs_agg, on=['org', 'year'])

In [12]:
org_year.groupby(['year', 'rank'])['message_logs'].agg(['mean', 'median']).reset_index().pivot(index='year', columns='rank', values=['mean', 'median'])

mean                       median           
rank       250k       500k        50k   250k  500k  50k
year                                                   
2015   3.000000  34.714286   6.750000    3.0   6.0  3.0
2016  22.545455  19.126126  12.098039    3.0   4.0  3.0
2017   4.551724   8.659794   4.981651    2.0   4.0  2.0
2018   6.320988  10.928870   5.781250    2.0   6.0  4.0
2019   6.867347  12.521073   6.269231    4.0   4.0  4.0
2020   5.233129  23.809969   3.886667    3.0   4.0  2.0
2021  28.604938  77.755725  22.253235    9.0  11.0  5.0
2022  21.959064  40.257463  15.652174    4.0   6.0  3.0

# lists

In [13]:
q = "select id, org, created, message from logs where message like '%recipient%'"
df_lists = redshift_query_read(q, schema='production')
df_lists['year'] = pd.to_datetime(df_lists['created']).dt.year
lists_agg = df_lists.groupby(['org', 'year'])['id'].count().reset_index()
lists_agg.columns = ['org', 'year', 'lists_logs']

In [14]:
org_year = org_year.merge(lists_agg, on=['org', 'year'])

In [16]:
org_year.groupby(['year', 'rank'])['lists_logs'].agg(['mean', 'median']).reset_index().pivot(index='year', columns='rank', values=['mean', 'median'])

mean                      median          
rank      250k       500k       50k   250k 500k  50k
year                                                
2015  3.000000   7.800000  4.000000    3.0  6.0  3.0
2016  9.000000  11.614458  6.222222    5.0  7.0  5.0
2017  7.694444  12.627586  7.652174    6.0  6.0  6.0
2018  7.880000  13.811321  7.333333    6.0  8.0  4.0
2019  7.639344  12.868421  6.346939    4.0  8.0  4.0
2020  3.446809   6.181034  3.433526    2.0  3.0  2.0
2021  3.737226   5.992337  3.067669    2.0  3.0  2.0
2022  2.137255   3.141509  1.976744    1.0  2.0  1.0